# Manipulate data the MXNet way with NDArray

MXNet's NDArray provides a data structure similar to NumPy's multi-dimensional array, adding some key capabilities. First, NDArrays support asynchronous computation on CPU, GPU and distributed cloud architectures. Second, they provide support for automatic differentiation. These properties make it an ideal library for machine learning both for research projects and production systems.


## Getting started

First, let's import ``mxnet`` and (for convenience) ``mxnet.ndarray``, the only dependencies we'll need in this tutorial.

In [9]:
import mxnet as mx
import mxnet.ndarray as nd

Next, let's see how to create an NDArray, without initializing values. Speficially we'll create a 2D array (also called a *matrix*) with 6 rows and 4 columns.

In [10]:
x = nd.empty(shape=(6,4))
print(x)

<NDArray 6x4 @cpu(0)>


Often we'll want create arrays whose values are sampled randomly. This is especially common when we intend to use the array as a parameter in a neural network. In this snipped, we initialize with values drawn from a standard normal distribution.

In [11]:
x1 = mx.nd.normal(loc=0, scale=1,shape=(6,4))
print(x1)
x = mx.nd.ones((6, 4), mx.cpu())                  # imperative style, 2-D arrays of '1's'
print x.asnumpy()   

<NDArray 6x4 @cpu(0)>
[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]


As in NumPy, the dimensions of each NDArray are accessible via the ``.shape`` attribute.

In [12]:
print(x.shape)

(6L, 4L)


We can also query its size, which is equal to the product of the components of the shape. Together with the precision of the stored values, this tells us how much memory the array occupies.

In [13]:
print(x.size)

24


## Operations

NDarray supports a large number of standard mathematical operations. 

In [14]:
print(x)
# y = nd.normal(shape=(6,4))
y = mx.nd.ones((6, 4), mx.cpu())  
print (y)
c = x + y
print(c)

<NDArray 6x4 @cpu(0)>
<NDArray 6x4 @cpu(0)>
<NDArray 6x4 @cpu(0)>


## In-place operations

In the previous example, we allocated new memory for the sum ``x+y`` and assigned a reference to the variable ``c``. To make better use of memory, we often prefer to perform operations in place, reusing already allocated memory. 

In MXNet, we can specify where to write the results of operations by assigning them with slice notation, e.g., ``result[:] = ...``.

In [15]:
result = nd.zeros(shape=(6,4))
result[:] = x+y
print(result)

<NDArray 6x4 @cpu(0)>


If we're not planning to re-use ``x``, then we can assign the result to ``x`` itself.

In [16]:
x[:] = x + y

But be careful! This is **NOT** the same as ``x = x + y``. If we don't use slice notation then we allocate new memory and assign a reference to the new data to the variable ``x``.

## Slicing

MXNet NDArrays support slicing in all the ridiculous ways you might imagine accessing Here's an example of reading the second and third rows from ``x``.

In [17]:
x[2:4]

<NDArray 2x4 @cpu(0)>

In [18]:
Now let's try whiting to a specific element.

SyntaxError: invalid syntax (<ipython-input-18-27a02bb56a70>, line 1)

In [ ]:
x[3,2] = 9.0
print(x[3])

## Weird multi-dimensional slicing

We can even write to arbitrary ranges along each of the axes.

In [ ]:
x[2:4,1:3] = 5.0
print(x)

## Converting from MXNet NDArray to NumPy 

Converting MXNet NDArrays to and from NumPy is easy. Note that, unlike in PyTorch, the converted arrays do not share memory.

In [ ]:
a = nd.ones(shape=(5))
print(a)

In [ ]:
b = a.asnumpy()
print(b)

In [ ]:
b[0] = 2
print(b)
print(a)

## Converting from NumPy Array to MXNet NDArray

Constructing an MXNet NDarray from a NumPy Array is straightforward.

In [ ]:
c = nd.array(b)
print(c)

## Managing context

In MXNet, every array has a context. A context could be the CPU, or one of many GPUs. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. 


In [ ]:
d = nd.array(b, mx.cpu())

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [ ]:
e = d.copyto(mx.cpu(1))
print(e)

## Watch out!

Imagine that your variable ``d`` already lives on your second GPU (``mx.gpu(1)``). What happens if we call ``d.copyto(mx.gpu(1))``? It will make a copy and allocate new memory, even though that variable already lives on the desired device! 

Often, we only want to make a copy if the variable *currently* lives in the wrong context. In these cases, we can call ``as_in_context()``. If the variable is already on ``mx.gpu(1)`` then this is a no-op.

In [ ]:
f = d.as_in_context(mx.cpu(0))
print(f)